<a href="https://colab.research.google.com/github/merucode/RL/blob/35-Study-Doc-Custom_grid_world/02_Reinforce.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
!git clone -b 35-Study-Doc-Custom_grid_world https://github.com/merucode/RL.git
!cd RL && mv * ../

Cloning into 'RL'...
remote: Enumerating objects: 201, done.
remote: Counting objects: 100% (121/121), done.
remote: Compressing objects: 100% (120/120), done.
remote: Total 201 (delta 47), reused 0 (delta 0), pack-reused 80
Receiving objects: 100% (201/201), 11.42 MiB | 5.76 MiB/s, done.
Resolving deltas: 100% (74/74), done.


In [6]:
!pip install -e gym-examples

Obtaining file:///content/gym-examples
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 710.3/710.3 kB 8.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 65.7 MB/s eta 0:00:00
  Created wheel for gym: filename=gym-0.26.0-py3-none-any.whl size=826275 sha256=8559d964befd9b4e09df87cbe477e46405b10474e73a79df39c9a503de936105
  Stored in directory: /root/.cache/pip/wheels/cd/da/7c/a034980e59e34687bfb6cfa3a5f3e9b389264a0c59519c59fa
Successfully built gym
  Attempting uninstall: pygame
    Found existing installation: pygame 2.5.0
    Uninstalling pygame-2.5.0:
      Successfully uninstalled pygame-2.5.0
  Attempting uninstall: gym
    Found existing installation: gym 0.25.2
    Uninstalling gym-0.25.2:
      Successfully uninstalled gym-0.25.2
  Running setup.py develop for gym-examples
ERROR: p

### NOTE: After inistall gym-examples, Restart runtime

In [ ]:
import time
import matplotlib.pyplot as plt
import gym
import numpy as np

In [ ]:
env = gym.make("gym_examples:gym_examples/GridWorld-v0")

SyntaxError: ignored

In [ ]:
# single env
print(f'observation space : {env.observation_space}')
print(f'action space      : {env.action_space}')
print(f'action shape      : {env.action_space.n}')

observation space : Dict('agent': Box(0, 4, (2,), int64), 'target': Box(0, 4, (2,), int64))
action space      : Discrete(4)
action shape      : 4


In [ ]:
obs, _ = env.reset()
print(obs)

obs = np.hstack((obs['agent'], obs['target']))
print(obs)

{'agent': array([4, 2]), 'target': array([1, 2])}
[4 2 1 2]


In [ ]:
action = env.action_space.sample()
env.step(0)

({'agent': array([3, 2]), 'target': array([3, 0])},
 0,
 False,
 False,
 {'distance': 2.0})

# Reinforce model

In [1]:
import gym
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.distributions import Categorical
import numpy as np

#Device
device = device = 'cuda' if torch.cuda.is_available() else 'cpu'

#Hyperparameters
learning_rate = 0.0002
gamma         = 0.98

class Policy(nn.Module):
    def __init__(self):
        super(Policy, self).__init__()
        self.data = []

        self.fc1 = nn.Linear(4, 128)
        self.fc2 = nn.Linear(128, 4)
        self.optimizer = optim.Adam(self.parameters(), lr=learning_rate)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.softmax(self.fc2(x), dim=0)
        return x

    def put_data(self, item):
        self.data.append(item)

    def train_net(self):
        R = 0
        self.optimizer.zero_grad()
        for r, prob in self.data[::-1]:
            R = r + gamma * R
            loss = -torch.log(prob) * R
            loss.backward()
        self.optimizer.step()
        self.data = []

In [2]:
def main():
    #env = gym.make('CartPole-v1')
    env = gym.make("gym_examples:gym_examples/GridWorld-v0")
    pi = Policy()
    score = 0.0
    max_score = 0
    print_interval = 100

    pi.to(device)

    for n_epi in range(6000):
        #s, _ = env.reset()
        obs, _ = env.reset()
        s = np.hstack((obs['agent'], obs['target']))
        done = False

        while not done:
            prob = pi(torch.from_numpy(s).float().to(device))
            m = Categorical(prob)
            a = m.sample()
            obs_next, r, done, truncated, info = env.step(a.item())
            s_prime =  np.hstack((obs_next['agent'], obs_next['target']))
            pi.put_data((r,prob[a]))
            s = s_prime
            score += r

        pi.train_net()

        if n_epi%print_interval==0 and n_epi!=0:
            print("# of episode :{}, avg score : {}".format(n_epi, score/print_interval))
            if score > max_score:
                torch.save(pi.state_dict(), 'model_weights.pth')
                print(f"# max score : {score/print_interval}, and save model_weights.pth")
                max_score = score
            score = 0.0

    env.close()

In [3]:
if __name__ == '__main__':
    main()

# of episode :100, avg score : 0.3325999999999773
# max score : 0.3325999999999773, and save model_weights.pth
# of episode :200, avg score : 0.44220000000000603
# max score : 0.44220000000000603, and save model_weights.pth
# of episode :300, avg score : 0.5127000000000118
# max score : 0.5127000000000118, and save model_weights.pth
# of episode :400, avg score : 0.5923000000000227
# max score : 0.5923000000000227, and save model_weights.pth
# of episode :500, avg score : 0.6982000000000076
# max score : 0.6982000000000076, and save model_weights.pth
# of episode :600, avg score : 0.7230000000000152
# max score : 0.7230000000000152, and save model_weights.pth
# of episode :700, avg score : 0.7556999999999993
# max score : 0.7556999999999993, and save model_weights.pth
# of episode :800, avg score : 0.7750999999999942
# max score : 0.7750999999999942, and save model_weights.pth
# of episode :900, avg score : 0.8051999999999896
# max score : 0.8051999999999896, and save model_weights.pth

# Render

In [19]:
env = gym.make('gym_examples:gym_examples/GridWorld-v0')

video_every=1

# define a trigger function, return True to start recording a new video:
episode_trigger=lambda episode_id: ((episode_id)%video_every==0)

# decorate env with a video recorder:
env = gym.wrappers.RecordVideo(env, "./video", episode_trigger=episode_trigger, video_length=100000)

def model_render():

    pi = Policy()
    pi.load_state_dict(torch.load('model_weights.pth'))
    pi.eval()
    pi.to(device)

    obs, _ = env.reset()
    s = np.hstack((obs['agent'], obs['target']))
    done = False
    score = 0

    while not done:
            prob = pi(torch.from_numpy(s).float().to(device).to(device))
            m = Categorical(prob)
            a = m.sample()
            obs_next, r, done, truncated, info = env.step(a.item())
            s_prime =  np.hstack((obs_next['agent'], obs_next['target']))
            score += r
        #print(f'# test episode {episode} score : {score}')
    env.close()

    # while not done:
    #     a = q.sample_action(torch.from_numpy(s).float(), epsilon=0)
    #     s_prime, r, done, _ = env.step(a)
    #     done_mask = 0.0 if done else 1.0
    #     s = s_prime
    #     score += r

    # print(score)
    # env.close()

/usr/local/lib/python3.10/dist-packages/gym/wrappers/record_video.py:75: UserWarning: WARN: Overwriting existing videos at /content/video folder (try specifying a different `video_folder` for the `RecordVideo` wrapper if this is not desired)
  logger.warn(


In [20]:
model_render()

/usr/local/lib/python3.10/dist-packages/gym/wrappers/monitoring/video_recorder.py:59: UserWarning: WARN: Disabling video recorder because environment <TimeLimit<OrderEnforcing<PassiveEnvChecker<GridWorldEnv<gym_examples/GridWorld-v0>>>>> was not initialized with any compatible video mode between `rgb_array` and `rgb_array_list`
  logger.warn(


In [15]:
from gym.wrappers import RecordVideo

#env = gym.make('CartPole-v1')

video_every=1

# define a trigger function, return True to start recording a new video:
episode_trigger=lambda episode_id: ((episode_id)%video_every==0)
# episode_trigger=lambda e: True
# decorate env with a video recorder:
#env = gym.wrappers.RecordVideo(env, "./video", episode_trigger=episode_trigger, video_length=100000)

#Device
device = device = 'cuda' if torch.cuda.is_available() else 'cpu'

def test_env(env_name, policy):
    env = gym.make(env_name)

    video_every=1
    episode_trigger=lambda episode_id: ((episode_id)%video_every==0)
    env = RecordVideo(env, "./video", episode_trigger=episode_trigger)
    pi = policy
    pi.load_state_dict(torch.load('model_weights.pth'))
    pi.eval()
    pi.to(device)

    for episode in range(10):
        obs, _ = env.reset()
        s = np.hstack((obs['agent'], obs['target']))
        score = 0
        done = False

        while not done:
            prob = pi(torch.from_numpy(s).float().to(device).to(device))
            m = Categorical(prob)
            a = m.sample()
            obs_next, r, done, truncated, info = env.step(a.item())
            s_prime =  np.hstack((obs_next['agent'], obs_next['target']))
            score += r
        print(f'# test episode {episode} score : {score}')
    env.close()
    #del env


In [4]:
# from gym.wrappers import RecordVideo
# from IPython.display import HTML
# from base64 import b64encode

# def test_env(env_name, policy): #, obs_rms
#     env = gym.make(env_name)
#     env = RecordVideo(env, 'videos', episode_trigger=lambda e: True)
#     pi = policy
#     pi.to(device)

#     for episode in range(10):
#         obs, _ = env.reset()
#         s = np.hstack((obs['agent'], obs['target']))
#         score = 0
#         done = False

#         while not done:
#             prob = pi(torch.from_numpy(s).float().to(device))
#             m = Categorical(prob)
#             a = m.sample()
#             obs_next, r, done, truncated, info = env.step(a.item())
#             s_prime =  np.hstack((obs_next['agent'], obs_next['target']))
#             score += r
#         print(f'episode {episode} score : {score}')
#     del env

In [16]:
test_env("gym_examples:gym_examples/GridWorld-v0", Policy())

# test episode 0 score : 0.97
# test episode 1 score : 0.98
# test episode 2 score : 0.97
# test episode 3 score : 0.97
# test episode 4 score : 0.92
# test episode 5 score : 0.88
# test episode 6 score : 0.96
# test episode 7 score : 1
# test episode 8 score : 0.9299999999999999
# test episode 9 score : 0.45999999999999974


/usr/local/lib/python3.10/dist-packages/gym/wrappers/monitoring/video_recorder.py:59: UserWarning: WARN: Disabling video recorder because environment <TimeLimit<OrderEnforcing<PassiveEnvChecker<GridWorldEnv<gym_examples/GridWorld-v0>>>>> was not initialized with any compatible video mode between `rgb_array` and `rgb_array_list`
  logger.warn(


In [17]:
from IPython.display import HTML
from base64 import b64encode

def display_video(episode=0):
    mp4 = open(f'./video/rl-video-episode-{episode}.mp4','rb').read()
    data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
    HTML("""
    <video width=500 controls>
          <source src="%s" type="video/mp4">
    </video>
    """ % data_url)

In [18]:
display_video()

FileNotFoundError: ignored